## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-13-22-39-48 +0000,nyt,Cuban Dissident Leader Leaves Prison for Exile...,https://www.nytimes.com/2025/10/13/world/ameri...
1,2025-10-13-22-39-07 +0000,nypost,Jimmy Breslin’s son slams AI-powered ‘writing ...,https://nypost.com/2025/10/13/us-news/jimmy-br...
2,2025-10-13-22-36-59 +0000,nypost,Trump said he would be ‘proud’ to visit Gaza —...,https://nypost.com/2025/10/13/world-news/trump...
3,2025-10-13-22-36-59 +0000,nyt,Live Updates: Hostages and Palestinian Prisone...,https://www.nytimes.com/live/2025/10/12/world/...
4,2025-10-13-22-36-54 +0000,nypost,"German woman who stole 2,400-year-old Greek re...",https://nypost.com/2025/10/13/world-news/germa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2332/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,57
36,hostages,22
290,china,22
30,gaza,21
108,israel,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
54,2025-10-13-20-53-37 +0000,wapo,Trump and world leaders meet in Egypt on futur...,https://www.washingtonpost.com/world/2025/10/1...,146
51,2025-10-13-20-57-45 +0000,bbc,Israeli hostages and Palestinian detainees fre...,https://www.bbc.com/news/articles/c740jx07vz0o...,130
62,2025-10-13-20-45-44 +0000,nypost,Dems largely quiet as Trump headlines historic...,https://nypost.com/2025/10/13/us-news/top-dems...,129
111,2025-10-13-17-58-10 +0000,nypost,Trump boasts Gaza peace process could be presi...,https://nypost.com/2025/10/13/us-news/trump-sa...,127
98,2025-10-13-18-40-17 +0000,nypost,Trump trolls Norway rep over Nobel Peace Prize...,https://nypost.com/2025/10/13/us-news/trump-tr...,124


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
54,146,2025-10-13-20-53-37 +0000,wapo,Trump and world leaders meet in Egypt on futur...,https://www.washingtonpost.com/world/2025/10/1...
220,66,2025-10-13-10-31-45 +0000,nypost,"Israeli twin brothers Ziv, Gali Berman reunite...",https://nypost.com/2025/10/13/world-news/israe...
79,56,2025-10-13-19-56-46 +0000,nypost,My mom was an NYPD cop who gave her life for N...,https://nypost.com/2025/10/13/lifestyle/my-mom...
185,55,2025-10-13-13-11-27 +0000,nypost,"Dow soars nearly 600 points after Trump, Vance...",https://nypost.com/2025/10/13/business/dow-fut...
93,54,2025-10-13-19-03-00 +0000,wsj,"Joel Mokyr, Philippe Aghion and Peter Howitt w...",https://www.wsj.com/economy/nobel-economics-pr...
19,51,2025-10-13-21-59-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...
61,44,2025-10-13-20-47-44 +0000,nypost,NYC bodega clerk locked up for infamous 1979 E...,https://nypost.com/2025/10/13/us-news/nyc-bode...
66,43,2025-10-13-20-33-00 +0000,wsj,"Fresh From Middle East Victory, Trump Sets His...",https://www.wsj.com/world/trump-ukraine-russia...
64,37,2025-10-13-20-39-04 +0000,nypost,NJ Gov. Murphy’s camp silent as critics rip hi...,https://nypost.com/2025/10/13/us-news/nj-gov-m...
150,36,2025-10-13-16-00-00 +0000,wsj,The relationship between capitalism and democr...,https://www.wsj.com/politics/united-states-ame...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
